<a href="https://colab.research.google.com/github/vfrantc/trans_experiments/blob/main/processing_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi 

Sun Jun  5 17:48:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install tqdm
!pip install thop
!pip install kornia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 512 kB 14.2 MB/s 


In [5]:
import os 
import cv2
from glob import glob 
from tqdm.notebook import tqdm 

In [3]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


# Run Transweather

In [4]:
!git clone https://github.com/vfrantc/TransMod.git
%cd TransMod
!mkdir -p data/test
!cp /content/drive/MyDrive/deweather/input.zip .
!unzip input.zip

Cloning into 'TransMod'...
remote: Enumerating objects: 423, done.
remote: Counting objects: 100% (159/159), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 423 (delta 103), reused 84 (delta 48), pack-reused 264
Receiving objects: 100% (423/423), 11.13 MiB | 2.48 MiB/s, done.
Resolving deltas: 100% (224/224), done.
/content/TransMod
Archive:  input.zip
   creating: input/
  inflating: input/get_list.py.ipynb  
  inflating: input/.DS_Store         
  inflating: __MACOSX/input/._.DS_Store  
  inflating: input/natural.txt       
   creating: input/input/
   creating: input/.ipynb_checkpoints/
  inflating: input/input/348.png     
  inflating: input/input/412.png     
  inflating: input/input/374.png     
  inflating: input/input/360.png     
  inflating: input/input/406.png     
  inflating: input/input/638.png     
  inflating: input/input/176.png     
  inflating: input/input/610.png     
  inflating: input/input/604.png     
  inflating: input/input/162.png     


In [6]:
for fname in tqdm(glob('input/input/*.png')):
  image = cv2.imread(fname)
  h, w = image.shape[:2]
  if h > w:
      image = cv2.resize(image, (int(w * 640 / h), 640))
  else:
      image = cv2.resize(image, (640, int(h * 640 / w)))

  cv2.imwrite(fname, image)

  0%|          | 0/497 [00:00<?, ?it/s]

In [7]:
!cp -r input data/test/
!cp input/natural.txt data/test/input.txt
!mkdir -p data/test/gt
!cp -r input/input data/test/gt/gt

In [8]:
!rm -rf Transweather
!cp /content/drive/MyDrive/derain/Transweather_orig.zip .
!unzip Transweather_orig.zip
!python testme.py -exp_name=./Transweather

Archive:  Transweather_orig.zip
   creating: Transweather/
  inflating: Transweather/latest     
  inflating: Transweather/best       
Seed:	19
cuda
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
--- Testing starts! ---
348.png
412.png
374.png
360.png
406.png
638.png
176.png
610.png
604.png
162.png
002.png
016.png
228.png
566.png
200.png
214.png
572.png
599.png
215.png
573.png
567.png
201.png
229.png
017.png
003.png
605.png
163.png
177.png
611.png
639.png
407.png
413.png
375.png
349.png
363.png
377.png
411.png
149.png
161.png
607.png
613.png
175.png
015.png
001.png
559.png
571.png
203

In [ ]:
!mkdir -p /drive/data
!cp -r data/test/input/input /drive/data
!cp -r ./results/natural/Transweather /drive/data/deweather 

In [10]:
%cd ..

/content


# Run retinexnet 

In [11]:
!ls

drive  sample_data  TransMod


In [ ]:
!git clone https://github.com/aasharma90/RetinexNet_PyTorch.git
%cd RetinexNet_PyTorch/

In [ ]:
!mkdir -p /drive/data/retinexnet
!python predict.py --data_dir /drive/data/deweather --res_dir /drive/data/retinexnet

# Run median + sharpening

In [9]:
%cd ..

/content


In [10]:
import cv2
import os
from glob import glob 
from tqdm.notebook import tqdm

In [ ]:
#!cp -r TransMod/data/test/input/input /drive/data

In [11]:
!mkdir -p /drive/data/postprocessed

In [12]:
def unsharp_mask(image, kernel_size=(5, 5), sigma=1.0, amount=1.0, threshold=0):
    """Return a sharpened version of the image, using an unsharp mask."""
    # https://en.wikipedia.org/wiki/Unsharp_masking
  
    blurred = cv2.GaussianBlur(image, kernel_size, sigma)
    sharpened = float(amount + 1) * image - float(amount) * blurred
    sharpened = np.maximum(sharpened, np.zeros(sharpened.shape))
    sharpened = np.minimum(sharpened, 255 * np.ones(sharpened.shape))
    sharpened = sharpened.round().astype(np.uint8)
    if threshold > 0:
        low_contrast_mask = np.absolute(image - blurred) < threshold
        np.copyto(sharpened, image, where=low_contrast_mask)
    return sharpened

In [ ]:
out_dir = '/drive/data/postprocessed'
for fname in tqdm('/drive/data/retinexnet/*.png'):
  imname = os.path.basename(fname)
  image = cv2.imread(fname)
  image = cv2.medianBlur(image, 5)  
  image = unsharp_mask(image)
  cv2.imwrite(os.path.join(out_dir, fname), image)

# Join for the presentation

In [ ]:
os.listdir('drive')